In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly
import scipy.signal as ss
import math
from selfpkg import readDB, MinToMS

In [5]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [6]:
def flapperOrientationCalculation(df, time):
    time = MinToMS(time, df)
    index = df.index[df['time'] == time].tolist()[0]
    
    x = movingaverage(df.accX, 5)[index]
    y = movingaverage(df.accY, 5)[index]
    
    while x > 98:
        x = movingaverage(df.accX, 5)[index-5]
    while y > 98:
        y = movingaverage(df.accX, 5)[index-5]
        
    angleX = math.acos(x/98) / 3.14 * 180
    angleY = math.acos(y/98) / 3.14 * 180
    
    if angleY >= 0 and angleY <= 90:
        if angleX >= 90:
            orientation = 90 - angleY
        else:
            orientation = 90 + angleY
    else:
        if angleX >= 90:
            orientation = 270 + (180 - angleY)
        else:
            orientation = 270 - (180 - angleY)
            
    return orientation

In [15]:
def flapperOrientationChart(df, time, orientation, flapperConfig, filename='flapperOrientationChart'):
    
    data = []
    #shadow part
    for i in range(8):
        trace0_range = go.Scatterpolar( 
            r = [0, 1, 1, 1, 1, 1, 0],
            theta = [0, orientation[i]-22.5, orientation[i]-11.25, orientation[i], orientation[i]+11.25, orientation[i]+22.5, 0],
            mode = 'lines',
            fill = 'toself',
            fillcolor = 'rgb(225, 225, 225)',
            name = "flapper" + str(flapperConfig[i]) + 'range',
            line = dict(
                color = 'rgb(225, 225, 225)'
            )
        )
        
        data.append(trace0_range)
    #flapper 
    for i in range(8):
        if i < 4:
            trace0 = go.Scatterpolar( 
                r = [0, 1],
                theta = [orientation[i], orientation[i]],
                mode = 'lines',
                name = "flapper" + str(flapperConfig[i]),
                line = dict(
                    color = 'rgb(225, 0, 0)'
                )
            )
            
            data.append(trace0)
        else:               
            trace4 = go.Scatterpolar( 
                r = [0, 1],
                theta = [orientation[i], orientation[i]],
                mode = 'lines',
                name = "flapper" + str(flapperConfig[i]),
                line = dict(
                    color = 'rgb(0, 0, 225)'
                )
            )
            
            data.append(trace4)
    #Gravity
    traceG = go.Scatterpolar( 
        r = [0, 1],
        theta = [270, 270],
        mode = 'lines',
        name = 'G',
        line = dict(
            color = 'rgb(25, 25, 25)',
            width = 4
        )
    )
    
    data.append(traceG)
    
    #x & y axis
    traceX = go.Scatterpolar(
        r = [0, 1],
        theta = [orientation[1], orientation[1]], #x axis == flapper-left
        mode = 'lines',
        name = 'X axis',
        line = dict(
            color = 'rgb(0, 255, 255)'
        )
    )
    
    traceY = go.Scatterpolar(
        r = [0, 1],
        theta = [orientation[2], orientation[2]], # y axis = flapper-gravity
        mode = 'lines',
        name = 'Y axis',
        line = dict(
            color = 'rgb(255, 0, 255)'
        )
    )
    
    data.append(traceX)
    data.append(traceY)
    
    #motion orientation by acc
    accX = df.accX
    accY = df.accY
    
    windowSize = 50
    
    avgX = movingaverage(accX, windowSize)
    avgY = movingaverage(accY, windowSize)
    
    threshold = 20
    timeRange = 0.01
    for i in range(len(accX)):
        accX[i] = accX[i] - avgX[i]
        accY[i] = accY[i] - avgY[i]
    for i in range(len(accX)):
        if accX[i] < threshold and accX[i] > -threshold:
            accX[i] = 0
        if accY[i] < threshold and accY[i] > -threshold:
            accY[i] = 0
    
    start = time - timeRange
    end = time + timeRange
    start = MinToMS(start, df)
    end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    accX = accX[startIndex: endIndex].tolist()
    accY = accY[startIndex: endIndex].tolist()
    
    accSum = []
    accOrientation = []
    for i in range(len(accX)):
        if accX[i] == 0 and accY[i] == 0:
            continue
        else:
            tmp = ((accX[i] ** 2 + accY[i] ** 2) ** (1/2))
            accSum.append(tmp)
            degreeX = math.acos(accX[i] / tmp) / 3.14 * 180
            degreeY = math.acos(accY[i] / tmp) / 3.14 * 180

            accOrientation.append(findCorrectAngle(orientation[1], orientation[2], degreeX, degreeY))
    
    for i in range(len(accOrientation)):
        while accOrientation[i] > 360:
            accOrientation[i] = accOrientation[i] - 360
    
    if len(accSum) != 0: #there exists an acc peak
        maxAcc = accSum[0]#max(accSum)
        accOrientation = [accOrientation[0]]
        for i in range(len(accOrientation)):
            traceAcc = go.Scatterpolar(
                r = [0, accSum[i]/maxAcc],
                theta = [accOrientation[i], accOrientation[i]],
                mode = 'lines',
                name = 'acceleration' + str(i),
                line = dict(
                    color = 'rgb(' + str(255 - 25 * i) + ', ' + str(255 - 25 * i) + ', 0)'
                )
            )
            data.append(traceAcc)
        
    #motion orientation by rotational speed
    rotationalSpeedX = df.rotationalSpeedX
    rotationalSpeedY = df.rotationalSpeedY
    
    threshold = 300
    timeRange = 0.02
    for i in range(len(accX)):
        if rotationalSpeedX[i] < threshold and rotationalSpeedX[i] > -threshold:
            rotationalSpeedX[i] = 0
        if rotationalSpeedY[i] < threshold and rotationalSpeedY[i] > -threshold:
            rotationalSpeedY[i] = 0
    
    start = time - timeRange
    end = time + timeRange
    start = MinToMS(start, df)
    end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    rotationalSpeedX = rotationalSpeedX[startIndex: endIndex].tolist()
    rotationalSpeedY = rotationalSpeedY[startIndex: endIndex].tolist()
    
    rotationalSpeedSum = []
    rotationalSpeedOrientation = []
    for i in range(len(rotationalSpeedX)):
        if rotationalSpeedX[i] == 0 and rotationalSpeedY[i] == 0:
            continue
        else:
            tmp = ((rotationalSpeedX[i] ** 2 + rotationalSpeedY[i] ** 2) ** (1/2))
            rotationalSpeedSum.append(tmp)
            degreeX = math.acos(rotationalSpeedX[i] / tmp) / 3.14 * 180
            degreeY = math.acos(rotationalSpeedY[i] / tmp) / 3.14 * 180

            rotationalSpeedOrientation.append(findCorrectAngleRotationalSpeed(orientation[1], orientation[2], degreeX, degreeY))
    
    for i in range(len(accOrientation)):
        while rotationalSpeedOrientation[i] > 360:
            rotationalSpeedOrientation[i] = rotationalSpeedOrientation[i] - 360
    
    if len(rotationalSpeedSum) != 0: #there exists a rotational speed peak
        maxRotationalSpeed = rotationalSpeedSum[0]#max(rotational speed sum)
        rotationalSpeedOrientation = [rotationalSpeedOrientation[0]]
        for i in range(len(rotationalSpeedOrientation)):
            traceRS = go.Scatterpolar(
                r = [0, rotationalSpeedSum[i]/maxRotationalSpeed],
                theta = [rotationalSpeedOrientation[i], rotationalSpeedOrientation[i]],
                mode = 'lines',
                name = 'Rotational Speed' + str(i),
                line = dict(
                    color = 'rgb(0, ' + str(255 - 25 * i) + ', 0)'
                )
            )
            data.append(traceRS)
    
    layout = go.Layout(
        title = filename,
        polar = dict(
            radialaxis = dict(
                visible = False,
                range = [0, 1]
            )
        )
    #     showlegend = False
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename=filename + '.html')
    
    return accOrientation

In [8]:
def findCorrectAngle(xAxis, yAxis, xDegree, yDegree):
    resX_0 = xAxis - xDegree
    resX_1 = xAxis + xDegree
    resY_0 = yAxis - yDegree
    resY_1 = yAxis + yDegree
    
    if abs(resX_0 - resY_1) < 0.1 or abs(resX_0 - resY_0) < 0.1:
        return resX_0
    else:
        return resX_1

In [9]:
def findCorrectAngleRotationalSpeed(xAxis, yAxis, yDegree, xDegree): #rotational speed has the reversed x,y axis
    resX_0 = xAxis - xDegree
    resX_1 = xAxis + xDegree
    resY_0 = yAxis - yDegree
    resY_1 = yAxis + yDegree
    
    if abs(resX_0 - resY_1) < 0.1 or abs(resX_0 - resY_0) < 0.1:
        return resX_0
    else:
        return resX_1

In [10]:
def getEachOrientation(init):
    orientation = []
    for i in range(8):
        if i <=3:
            orientation.append(init + i * 90)
        else:
            orientation.append(init + 45 + (i-4) * 90)
    return orientation

In [11]:
def flapperOrientation(df, time, flapperConfig, abFlapper=None, filename='flapperOrientation'):
    #calcualte the all flapper orientation
    initFlapper = flapperOrientationCalculation(df, time)
    orientation = getEachOrientation(initFlapper)
    for i in range(len(orientation)):
        if orientation[i] > 360:
            orientation[i] = orientation[i] - 360
    accOrientation = flapperOrientationChart(df, time, orientation, flapperConfig, filename)
    
    #compare the acc & abnormal Flapper orientation to see if it works as a POI/bump, optional
    if abFlapper == None:
        return
    for flapper in abFlapper:
        flapper_angle = orientation[flapperConfig.index(flapper)]
        acc_angle = accOrientation[0]
        diff = abs(flapper_angle - acc_angle)
        threshold = 50
        if flapper >= 0 and flapper <= 3: # if it is a flapper in front
            if diff < threshold or abs(diff - 360) < threshold:
                print('flapper' + str(flapper) + ' corresponds with acceleration @' + str(time) + 'min' + ', probably a leek')
            if abs(diff - 180) < threshold:
                print('flapper' + str(flapper) + ' corresponds with acceleration @' + str(time) + 'min' + ', probably a bump')
        else: # or back
            if abs(diff - 180) < threshold:
                print('flapper' + str(flapper) + ' corresponds with acceleration @' + str(time) + 'min' + ', probably a leek')
            if diff < threshold or abs(diff - 360) < threshold:
                print('flapper' + str(flapper) + ' corresponds with acceleration @' + str(time) + 'min' + ', probably a bump')
        # the front & back flapper have reversed feature, becasue our IMU sensor is in the middle.

In [16]:
time = 32
# abFlapper = [4]

run = 6
# df = pd.read_csv('/Users/meng/Documents/WatchTower/rawData/Wise County/processed Data/Wise_Run' + str(run) + '_Trimed.csv')
csv_database = 'sqlite:///cambridge02.db' 
df = readDB(csv_database)
# flapperConfig = [0, 3, 2, 1, 7, 6, 5, 4] # flapper Config
# flapperConfig = [2, 3, 1, 0, 7, 5, 4, 6]
flapperConfig = [3, 0, 1, 2, 4, 5, 6, 7]
flapperOrientation(df, time, flapperConfig, filename='../charts/flapperOrientation')

In [ ]:
##################################################################

In [62]:
#function for generating acceleration chart
def accChart(df, time, start=None, end=None, filename='accChart'):

    timeConversionVal = 60000
    
    accX = df.accX
    accY = df.accY
    
    windowSize = 50
    
    avgX = movingaverage(accX, windowSize)
    avgY = movingaverage(accY, windowSize)
    
    threshold = 20
    for i in range(len(accX)):
        accX[i] = accX[i] - avgX[i]
        accY[i] = accY[i] - avgY[i]
    for i in range(len(accX)):
        if accX[i] < threshold and accX[i] > -threshold:
            accX[i] = 0
        if accY[i] < threshold and accY[i] > -threshold:
            accY[i] = 0
    
    start = time - 0.01
    end = time + 0.01
    start = MinToMS(start, df)
    end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]

    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=accX[startIndex: endIndex],
        name = 'accX processed',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=accY[startIndex: endIndex],
        name = 'accY processed',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    data = [trace2, trace3]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Acceleration'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')
    
    return accX[startIndex: endIndex].tolist(), accY[startIndex: endIndex].tolist()

In [91]:
#function for generating acceleration chart
def rotationalSpeedChart(df, time, start=None, end=None, filename='rotationalSpeedChart'):

    timeConversionVal = 60000
    
    rotationalSpeedX = df.rotationalSpeedX
    rotationalSpeedY = df.rotationalSpeedY
    
    threshold = 300
    for i in range(len(rotationalSpeedX)):
        if rotationalSpeedX[i] < threshold and rotationalSpeedX[i] > -threshold:
            rotationalSpeedX[i] = 0
        if rotationalSpeedY[i] < threshold and rotationalSpeedY[i] > -threshold:
            rotationalSpeedY[i] = 0
    
    start = time - 0.02
    end = time + 0.02
    start = MinToMS(start, df)
    end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]

    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=rotationalSpeedX[startIndex: endIndex],
        name = 'rotationalSpeed X processed',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=rotationalSpeedY[startIndex: endIndex],
        name = 'rotationalSpeed Y processed',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    data = [trace2, trace3]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'rotationalSpeed'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')
    
    return rotationalSpeedX[startIndex: endIndex].tolist(), rotationalSpeedY[startIndex: endIndex].tolist()

In [92]:
run = 6
time = 6.41
df = pd.read_csv('/Users/meng/Documents/WatchTower/rawData/Wise County/processed Data/Wise_Run' + str(run) + '_Trimed.csv')
accX, accY = accChart(df, time, filename='testMotionDetection')

NameError: name 'rotationalSpeedX' is not defined

In [90]:
run = 6
time = 8.41
df = pd.read_csv('/Users/meng/Documents/WatchTower/rawData/Wise County/processed Data/Wise_Run' + str(run) + '_Trimed.csv')
accX, accY = rotationalSpeedChart(df, time, filename='testMotionDetection')

In [69]:
print(accX)
print(accY)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -321.0, -581.0, 832.0, 523.0, -736.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1399.0, 561.0, 0.0, 536.0, -709.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [80]:
acc = []
for i in range(len(accX)):
    if accX[i] == 0 and accY[i] == 0:
        continue
    else:
        acc.append((accX[i] ** 2 + accY[i] ** 2) ** (1/2))

In [81]:
print(acc)

[32.16, 43.980000000000004, 21.60000000000001]
